In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")  # specify to ignore warning messages
from statsmodels.tools.sm_exceptions import ConvergenceWarning
from statsmodels.tools.sm_exceptions import HessianInversionWarning
warnings.simplefilter('ignore', category=ConvergenceWarning)
warnings.simplefilter('ignore', category=HessianInversionWarning)

In [2]:
# 특정파일을 절대경로로 datafile read함.
df_train = pd.read_csv('funda_train.csv')
df_sub = pd.read_csv('submission.csv')

In [3]:
%%time
df_train['transaction_time'] = df_train['transacted_date'] + "-" + df_train['transacted_time']
df_train['transaction_time'] = pd.to_datetime(df_train['transaction_time'], format='%Y-%m-%d-%H:%M', infer_datetime_format=True)

Wall time: 16.8 s


In [4]:
df_train.drop(['transacted_date', 'transacted_time'], axis=1, inplace=True)

In [5]:
from datetime import datetime

In [6]:
def cancel_delete(df_):
    df = df_.copy(deep=True)
    store_id_lst = df.store_id.unique()
    del_idx = []
    extra_idx = []
    for s_id in (store_id_lst):
        tmp = df[(df['store_id'] == s_id)]
        minus_amount = tmp[tmp['amount'] < 0]
        for idx, col in minus_amount.iterrows():
            df_search = tmp[(tmp['amount'] == - col['amount']) &
                      (tmp['card_id'] == col['card_id']) &
                      (tmp['transaction_time'] <= col['transaction_time']) &
                      (tmp['card_company'] == col['card_company'])]
            if len(df_search) == 0:
        # 결제 취소했지만 데이터에 없는 경우 -> 추후 제거
                extra_idx.append(idx)
            else:
        # 가장 최근 결제 내역을 제거
                del_idx.append(df_search.index[-1])
                del_idx.append(idx)

    df.drop(del_idx, axis=0, inplace=True)
    df.drop(extra_idx, axis=0, inplace=True)
    return df

In [7]:
%%time
df_cleaned = cancel_delete(df_train)
# 총 매출이 0인 경우도 데이터에서 제거
df_cleaned.drop(df_cleaned[df_cleaned.amount <= 0].index, axis=0, inplace=True)

Wall time: 1min 27s


In [8]:
df_prep = df_cleaned.set_index('transaction_time')
df_prep.head()

,store_id,card_id,card_company,installment_term,region,type_of_business,amount
transaction_time,,,,,,,
2016-06-01 13:13:00,0,0,b,0,NaN,기타 미용업,1857.142857
2016-06-01 18:12:00,0,1,h,0,NaN,기타 미용업,857.142857
2016-06-01 18:52:00,0,2,c,0,NaN,기타 미용업,2000.000000
2016-06-01 20:22:00,0,3,a,0,NaN,기타 미용업,7857.142857
2016-06-02 11:06:00,0,4,c,0,NaN,기타 미용업,2000.000000


In [9]:
# 최근 데이터가 없는 상점을 찾음.
def no_data(df, at_least_need_date = '2019-01-01'):
    u_store_id = df['store_id'].unique()
    del_store_id_lst = []
    for s_id in u_store_id:
        df_tmp = df[df['store_id'] == s_id]
    # 최근 매출 날짜를 봄
        df_tmp = df_tmp.resample('M').sum().index[-1]
    # 2019년 1월 1일 이전이면 예측 0으로. 이 부분은 나중에 바꿔도 됨.
        if df_tmp < datetime.strptime(at_least_need_date, "%Y-%m-%d"):
            del_store_id_lst.append(s_id)
    return del_store_id_lst

In [10]:
%%time
no_data_store_id = no_data(df_prep)
no_data_store_id

Wall time: 17.2 s


[111,
 257,
 374,
 479,
 619,
 632,
 640,
 772,
 795,
 802,
 838,
 999,
 1217,
 1233,
 1520,
 1527,
 1567,
 1598,
 1604,
 1685,
 1967,
 2027,
 2053,
 2058,
 2086,
 2119]

In [11]:
# 전부 2018년도까지만 매출 데이터 있음, 2019년도 매출 데이터가 없음 
print(no_data_store_id)

[111, 257, 374, 479, 619, 632, 640, 772, 795, 802, 838, 999, 1217, 1233, 1520, 1527, 1567, 1598, 1604, 1685, 1967, 2027, 2053, 2058, 2086, 2119]


In [12]:
df_prep.fillna("NULL", inplace=True)

In [13]:
import itertools
from statsmodels.tsa.arima_model import ARIMA, ARMAResults
from statsmodels.tsa.stattools import adfuller
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [14]:
# stationary test
def adfTest(timeseries, diff):
    if diff == 0:
        result = adfuller(timeseries)
    else:
        diff_ts = timeseries.copy(deep=True)
        for d in range(1, diff+1):
            diff_ts = (diff_ts - diff_ts.shift(1)).dropna()
        result = adfuller(diff_ts)
    if result[0] < result[4]['1%'] :
        return True
    else:
        return False

# ARIMA
def ARIMA_fit(timeseries, s_id):
    df_ts = np.log(timeseries+1)
    df_ts = pd.Series([1 if i == 0 else i for i in df_ts ])
    df_ts_mean = np.mean(df_ts)
    df_ts_std = np.std(df_ts)
    df_ts = (df_ts - df_ts_mean)/df_ts_std
  # diff check
    diff = -1
    for i in range(0, 3):
        if adfTest(df_ts, i):
            diff = i

    if diff == -1:
        print("not stationary : ", s_id)
        return -1

    else:
        p = q = range(3)
        min_aic = 100000
        d = range(diff+1)
        min_param = []
        pdq = list(itertools.product(p, d, q))
    for param in pdq:
        try:
            mod = ARIMA(df_ts, order=(param[0], param[1], param[2]))
            results = mod.fit()
            if results.aic < min_aic:
                min_aic = results.aic
                min_param = [param[0], param[1], param[2]]
                final_model = results
        except:
            continue
    return sum(np.exp(final_model.forecast(steps=3)[0] * df_ts_std + df_ts_mean))

In [15]:
%%time
unique_store_id = df_prep['store_id'].unique()
unique_store_id = [i for i in unique_store_id if i not in no_data_store_id]
not_stationary_lst = []
random_state = 100
ts_predict = {}
for one_store_id in unique_store_id:
    if one_store_id % 100 == 0:
        print(one_store_id)
    df_month_sampling_amount = df_prep[df_prep['store_id'] == one_store_id]['amount'].resample('M').sum()
    arima_predict_ = ARIMA_fit(df_month_sampling_amount, one_store_id)
    if arima_predict_ == -1:
        not_stationary_lst.append(one_store_id)
    else:
        ts_predict[one_store_id] = arima_predict_
    
len(not_stationary_lst)

0
not stationary :  2
not stationary :  10
not stationary :  13
not stationary :  16
not stationary :  19
not stationary :  22
not stationary :  40
not stationary :  61
not stationary :  83
100
not stationary :  102
not stationary :  103
not stationary :  104
not stationary :  126
not stationary :  130
not stationary :  153
not stationary :  159
not stationary :  166
not stationary :  170
not stationary :  188
200
not stationary :  211
not stationary :  229
not stationary :  269
300
not stationary :  317
not stationary :  325
not stationary :  331
not stationary :  335
not stationary :  338
not stationary :  339
not stationary :  352
not stationary :  365
not stationary :  389
not stationary :  392
not stationary :  394
400
not stationary :  406
not stationary :  407
not stationary :  425
not stationary :  440
not stationary :  456
not stationary :  459
not stationary :  462
not stationary :  466
not stationary :  486
not stationary :  487
not stationary :  495
not stationary :  498
50

212

In [16]:
rf_score = {}
def rf_prediction(df_, forward_step, random_state):
  # 예측 값을 shift함.
    df_['predict_val'] = df_['amount'].shift(-forward_step)

    df_train = df_.iloc[:df_.shape[0]-1, :]
    df_train.dropna(inplace=True)
  # 마지막 value -> final predict
    final_predict_value = df_.iloc[df_.shape[0]-1,  df_.columns != 'predict_val' ]
    X = df_train.loc[:, df_.columns != 'predict_val']
    Y = df_train['predict_val']
  #x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=random_state)
  
    param_grid = {
        'bootstrap': [True],
        'max_depth': [2, 3, 4],
        'max_features': ['auto', 'sqrt', 4],
        'min_samples_leaf': [3, 4],
        'min_samples_split': [2, 4],
        'n_estimators': [3, 5, 7]
    }

    rf_model = RandomForestRegressor()
    grid_search = GridSearchCV(estimator = rf_model, param_grid = param_grid, scoring='neg_mean_absolute_error',
                          cv = 3, n_jobs = -1)
  
    grid_search.fit(X.values, Y.values)
    best_grid = grid_search.best_estimator_
    rf_score[s_id] = grid_search.best_score_
    return best_grid.predict([final_predict_value.values])


def df_replace_outlier(df_):
    cols = list(df_.columns)
    for col in cols:
        df_col_series = df_[col]
        qtile = df_col_series.quantile(0.2)
        for idx, val in df_col_series.items():
            if val <= qtile:
                df_.loc[idx][col] = qtile
    return df_


def preprocessed_for_rf(df_ts, feature_lags):
    month_sample_idx = df_ts.resample('M').sum().index
  
    df_result = pd.DataFrame(index=month_sample_idx)
  # 한달 결제 횟수
    df_result['pay_count'] = (df_ts['store_id'].resample('M').count()).fillna(0)

  # 월별 해당 지역에 대한 전체 결제 금액
    month_sampling_region = df_prep[df_prep['region'] == df_ts['region'][0]].resample('M')
    df_result['region_total_amount'] = month_sampling_region['amount'].sum()

  # 월별 해당 지역에서 결제한 가게의 수
    df_result['region_store_count'] = 1
    for idx in month_sample_idx:
        df_result.loc[idx, 'region_store_count'] = len(month_sampling_region.store_id.unique().loc[idx])
  
  # 월별 해당 지역에 대한 평균 매출 금액
    df_result['region_mean_amount'] = (df_result['region_total_amount'] / df_result['region_store_count']).fillna(0)

  # 월별 업종에 대한 전체 결제 금액
    month_sampling_business = df_prep[df_prep['type_of_business'] == df_ts['type_of_business'][0]].resample('M')
    df_result['business_total_amount'] = month_sampling_business['amount'].sum()

    df_result['business_store_count'] = 1
    for idx in month_sample_idx:
        df_result.loc[idx, 'business_store_count'] = len(month_sampling_business.store_id.unique().loc[idx])
  
    df_result['business_mean_amount'] = (df_result['business_total_amount'] / df_result['business_store_count']).fillna(0)
  
    del df_result['region_total_amount']
    del df_result['region_store_count']
    del df_result['business_total_amount']
    del df_result['business_store_count']

    df_result['amount'] = df_ts.resample('M')['amount'].sum()
  #print(df_result.columns)
    df_result = df_replace_outlier(df_result)
  
    cols = df_result.columns
  # lag변수 생성
    for col in cols:
        for lag in range(1, feature_lags + 1):
            df_result[col + "_lag_" + str(lag)] = df_result[col].shift(lag)
  #display(df_result)
    df_result.dropna(inplace=True)
    return df_result

In [17]:
%%time
random_state = 100
not_stationary_prediction = {}
not_stationary_store_id = not_stationary_lst

df_not_stationary = df_prep[df_prep['store_id'].isin(not_stationary_store_id)]

lag_num = 3
for idx, s_id in enumerate(not_stationary_store_id):
    if idx % 50 == 0:
        print(idx, s_id)
    df_tmp = df_not_stationary[df_not_stationary['store_id'] == s_id]
    df_train = preprocessed_for_rf(df_tmp, feature_lags = lag_num)
    predict_val = []
  # 3달 예측
    for d in range(1, 4):
        predict_val.append(rf_prediction(df_train, d, random_state))
    not_stationary_prediction[s_id] = sum(predict_val)

0 2
50 560
100 1025
150 1548
200 2036
Wall time: 25min 20s


In [18]:
%%time
random_state = 100
not_stationary_prediction = {}
not_stationary_store_id = not_stationary_lst

df_not_stationary = df_prep[df_prep['store_id'].isin(not_stationary_store_id)]

lag_num = 3
for idx, s_id in enumerate(not_stationary_store_id):
    if idx % 50 == 0:
        print(idx, s_id)
    df_tmp = df_not_stationary[df_not_stationary['store_id'] == s_id]
    df_train = preprocessed_for_rf(df_tmp, feature_lags = lag_num)
    predict_val = []
  # 3달 예측
    for d in range(1, 4):
        predict_val.append(rf_prediction(df_train, d, random_state))
    not_stationary_prediction[s_id] = sum(predict_val)

0 2
50 560
100 1025
150 1548
200 2036
Wall time: 41min 21s


In [20]:
df_sub_store_id_lst = df_sub['store_id'].unique()
for s_id in df_sub_store_id_lst:
    final_value = 0
    if s_id in ts_predict.keys():
        final_value = ts_predict[s_id]
    elif s_id in no_data_store_id:
        final_value = 0
    elif s_id in not_stationary_lst:
        final_value = not_stationary_prediction[s_id]
    else:
        print("no store id : ", s_id)
    df_sub.iloc[df_sub[df_sub['store_id'] == s_id].index[0], 1] = final_value
df_sub.to_csv('funda_sub.csv', index=False)

result?